<a href="https://colab.research.google.com/github/MohmadYaacoub/Data/blob/main/Data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import pandas as pd
import numpy as np

def data_generator(base_file_path, max_files_per_class=50):
    # Define the dictionary to store file paths for each category
    text_files = {
        "Agrapheuse Comb": [], "Apple": [], "Battery": [], "Book": [], "Bottle of water": [], "Brush teeth": [],
        "Calculator": [], "Car remote": [], "Clamp": [], "Coin": [], "Cup": [],
        "Forke": [], "Key": [], "keyboard": [], "Mouse": [], "Orange": [],
        "Pen": [], "Pepsi": [], "Phone Comb": [], "Plate": [], "Remote": [],
        "Scissors Comb": [], "Scotch Comb": [], "Screwdriver": [], "Shampo Comb": [], "Spoon": [], "Spray": [], "Umbrella": []
    }

    # Retrieve all directories in the base file path
    data = glob.glob(base_file_path + "/*")

    # Assign file paths to the respective categories
    for directory_path in data:
        for key in text_files.keys():
            if key in directory_path:
                text_files[key].append(directory_path)

    # Initialize lists to hold the data and labels
    X, y = [], []
    class_counts = {key: 0 for key in text_files.keys()}  # Initialize class counts

    # Process each category
    for label, (key, directory_paths) in enumerate(text_files.items(), start=0):
        for directory_path in directory_paths:
            files_in_directory = glob.glob(directory_path + "/*")
            for file_path in files_in_directory:
                df = pd.read_csv(file_path, header=None)
                df_array = df.iloc[1:55, :35].values
                X_text.append(df_array)
                y_text.append(label)  # Assign numeric label
                class_counts[key] += 1  # Increment class count

                # Break if reached the maximum number of files for this class
                if class_counts[key] == max_files_per_class:
                    break
            # Break if reached the maximum number of files for this class
            if class_counts[key] == max_files_per_class:
                break
        print(f"Class {key}: {class_counts[key]} files")

    # Convert lists to NumPy arrays
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y, dtype=np.int32)

    print("X shape:", X.shape)
    print("y shape:", y.shape)

    return X, y

# Example usage:
file_path = "C:/Users/mhamad.yackoub/OneDrive - unige.it/Desktop/VScode/Multimodal Glove/Csv Fusion"
X, y = data_generator(file_path, max_files_per_class=50)
